In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pygad
from utils import compute_metrics, roc_auc_score, compute_predicted_probabilities, compute_nri, make_prob_dict

global_fitness_history = {}

# ------------------------------------------------------------------------------
# 1. Set random seeds for reproducibility
# ------------------------------------------------------------------------------
np.random.seed(42)  # Seed for NumPy random operations

# ------------------------------------------------------------------------------
# 2. Load the dataset
# ------------------------------------------------------------------------------
full_data = pd.read_csv("lace.csv")
train_data, test_data = train_test_split(full_data, test_size=0.4, random_state=42, stratify = full_data['30Readmit'])

for mode in ["train","test","full"]:
    exec(f"{mode}_LOS = {mode}_data['Length of stay (days)'].values ")
    exec(f"{mode}_Acute = {mode}_data['Acute (emergent) admission'].values")
    exec(f"{mode}_Charlson = {mode}_data['Charlson Comorbidity Index'].values")
    exec(f"{mode}_ED = {mode}_data['Number of ED visits within 6 months'].values")
    exec(f"{mode}_y = {mode}_data['30Readmit'].values")
    
# ------------------------------------------------------------------------------
# 3. Define the original LACE scoring function (baseline)
# ------------------------------------------------------------------------------
W = [1,1,1,2,1,3,1,4,1,7,2,14,
     3,
     1,1,1,2,1,3,2,4,
     1,1,1,2,1,3,1,4]

def scoring(w_L1, t_L1, w_L2, t_L2, w_L3, t_L3, w_L4, t_L4, w_L5, t_L5, w_L6, t_L6, w_A, w_C1, t_C1, w_C2, t_C2, w_C3, t_C3, w_C4, t_C4, w_E1, t_E1, w_E2, t_E2, w_E3, t_E3, w_E4, t_E4, mode="train"):
    LOS = eval(f"{mode}_LOS")
    Acute = eval(f"{mode}_Acute")
    Charlson = eval(f"{mode}_Charlson")
    ED = eval(f"{mode}_ED")

    local_vars = dict(locals())
    score_LOS = sum( [eval(f"w_L{i} * (LOS>=t_L{i})", globals(), local_vars) for i in range(1,7) ] )
    score_Acute = w_A * (Acute > 0.5)
    score_Charlson = sum( [eval(f"w_C{i} * (Charlson>=t_C{i})", globals(), local_vars) for i in range(1,5) ] )
    score_ED = sum( [eval(f"w_E{i} * (ED>=t_E{i})", globals(), local_vars) for i in range(1,5) ] )
    total_score = score_LOS + score_Acute + score_Charlson + score_ED
    return total_score
    
# Compute the baseline (original) scores and their AUC
orig_scores = scoring(*W)
orig_auc = roc_auc_score(train_y, orig_scores)
test_orig_scores = scoring(*W, mode = "test")

# ------------------------------------------------------------------------------
# 4. Define the fitness function for candidate solutions
# ------------------------------------------------------------------------------
def fitness_func(ga_instance, solution, solution_idx):
    global global_fitness_history
    if tuple(solution) in global_fitness_history:
        return global_fitness_history[tuple(solution)]
        
    # Unpack genes
    w_L1, t_L1, w_L2, t_L2, w_L3, t_L3, w_L4, t_L4, w_L5, t_L5, w_L6, t_L6, w_A, w_C1, t_C1, w_C2, t_C2, w_C3, t_C3, w_C4, t_C4, w_E1, t_E1, w_E2, t_E2, w_E3, t_E3, w_E4, t_E4 = solution
    
    heavy_constant = 1  # Constant to heavily penalize ordering violations
    light_constant = 0.05 # Constant to lightly penalize deviation from original score 
    
    # Apply penalty only when the constraint is violated:
    penalty = heavy_constant * max(0, t_L1 - (t_L2-1))
    penalty += heavy_constant * max(0, t_L2 - (t_L3-1))
    penalty += heavy_constant * max(0, t_L3 - (t_L4-1))
    penalty += heavy_constant * max(0, t_L4 - (t_L5-1))
    penalty += heavy_constant * max(0, t_L4 - (t_L5-1))
    penalty += heavy_constant * max(0, t_L5 - (t_L6-1))
    penalty += heavy_constant * max(0, t_C1 - (t_C2-1))
    penalty += heavy_constant * max(0, t_C2 - (t_C3-1))
    penalty += heavy_constant * max(0, t_C3 - (t_C4-1))
    penalty += heavy_constant * max(0, t_E1 - (t_E2-1))
    penalty += heavy_constant * max(0, t_E2 - (t_E3-1))
    penalty += heavy_constant * max(0, t_E3 - (t_E4-1))
    penalty += light_constant * np.average([np.abs(i-j)/i for i,j in zip(W,solution)]) # Average value of the ratio of the change
    
    # Compute candidate's score using indicator functions:
    candidate_score = scoring(*solution)

    try:
        # --- Objective 1: Active AUC Improvement ---
        candidate_auc = roc_auc_score(train_y, candidate_score)
        active_auc = candidate_auc - orig_auc
        
        # --- Objective 2: Active Brier Score Improvement ---
        baseline_probs = compute_predicted_probabilities(orig_scores, train_y)
        candidate_probs = compute_predicted_probabilities(candidate_score, train_y)
        brier_candidate = np.mean((candidate_probs - np.array(train_y)) ** 2)
        brier_baseline = np.mean((baseline_probs - np.array(train_y)) ** 2)
        active_brier = brier_baseline - brier_candidate
        
        # --- Objective 3: Net Reclassification Improvement (NRI) ---
        nri = compute_nri(baseline_probs, candidate_probs, train_y)
        
        obj1 = max(0,active_auc - penalty)
        obj2 = max(0,active_brier - penalty)
        obj3 = 10**-6 if nri>0 else 0

    except ValueError:
        obj1, obj2, obj3 = -0.01, -0.01, -0.01
    
    # Store the fitness vector in the global dictionary.
    global_fitness_history[tuple(solution)] = np.array([obj1, obj2, obj3])
    
    # Return the vector of objectives.
    return np.array([obj1, obj2, obj3])

# ------------------------------------------------------------------------------
# 5. Define the gene space for integer solutions
# ------------------------------------------------------------------------------
 # Original score, with lower limit of 1 and upper limit that is 1.5x of the current value + 1 

gene_space = [list(range(1,i//2+i+1)) for i in W] 

# ------------------------------------------------------------------------------
# 6. Set up and run the Genetic Algorithm with PyGAD
# ------------------------------------------------------------------------------
ga_instance = pygad.GA(
    num_generations=100,         
    num_parents_mating=100,   
    fitness_func=fitness_func,   
    sol_per_pop=500,          
    num_genes=len(gene_space),  
    gene_space=gene_space,       
    mutation_probability=0.2,   
    random_seed=0,          
    initial_population = [W]*100,
    parent_selection_type='nsga2',
)

ga_instance.run()

# Retrieve the best solution
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Best solution (genes):", solution)

# ------------------------------------------------------------------------------
# 7. Report the AUC of the best candidate solution
# ------------------------------------------------------------------------------
# Recompute candidate score for the best solution

best_candidate_score = scoring(*solution)
train_baseline_prob_dict = make_prob_dict(orig_scores, train_y)
train_prob_dict = make_prob_dict(best_candidate_score, train_y)
test_best_candidate_score = scoring(*solution, mode = 'test')
_, train_threshold = compute_metrics(orig_scores, train_y, train_prob_dict=train_baseline_prob_dict)
_, new_train_threshold = compute_metrics(best_candidate_score, train_y, baseline_scores=orig_scores, train_prob_dict=train_prob_dict,train_baseline_prob_dict=train_baseline_prob_dict)

print("------------------------------------------------------------------------------\nOriginal Test Results\n------------------------------------------------------------------------------")
results = compute_metrics(test_orig_scores, test_y, train_threshold=train_threshold, baseline_scores=test_orig_scores, train_prob_dict=train_baseline_prob_dict, train_baseline_prob_dict=train_baseline_prob_dict)
for key, value in results.items():
    print(f"{key}: {value}")

print("------------------------------------------------------------------------------\nTest Results\n------------------------------------------------------------------------------")
results = compute_metrics(test_best_candidate_score, test_y, train_threshold=new_train_threshold, baseline_scores=test_orig_scores, train_prob_dict=train_prob_dict, train_baseline_prob_dict=train_baseline_prob_dict)
for key, value in results.items():
    print(f"{key}: {value}")

Best solution (genes): [ 1.  1.  1.  2.  1.  3.  1.  6.  1.  8.  1. 17.  2.  1.  1.  1.  2.  1.
  4.  3.  6.  1.  1.  1.  3.  1.  4.  1.  6.]
------------------------------------------------------------------------------
Original Test Results
------------------------------------------------------------------------------
AUC: 0.6764950166112956
Optimal Threshold: 10.0
Max Youden Index: 0.22093023255813948
Sensitivity: 1.0
Specificity: 0.22093023255813954
PPV: 0.0945945945945946
NPV: 1.0
Brier Score: 0.07304237491936502
ECE: 0.06926645102125673
Hosmer-Lemeshow p-value: 0.11452784320005827
NRI: 0.0
IDI: 0.0
------------------------------------------------------------------------------
Test Results
------------------------------------------------------------------------------
AUC: 0.6879152823920266
Optimal Threshold: 9.0
Max Youden Index: 0.34883720930232553
Sensitivity: 1.0
Specificity: 0.3488372093023256
PPV: 0.1111111111111111
NPV: 1.0
Brier Score: 0.06897768399502605
ECE: 0.0230915041